## Network Viability - Rewards and Fees

- **Goal**: to understand the impact(s) of 1) changing the servicer block reward allocation amount, and 2) changing the gateway/application fee amounts, on supply-side profitability under different demand-side growth regimes. 
- **System Goals Targeted**:
  - Economic Viability
  - Long-Term Value Generation
  - Accessibility
- **Design**: create a series of demand-side growth scenarios, and for a given network topology assess supply-side profitability
- **Testing mechanism**:
  - There is a growth process for relay demand
    - For a single service ("one chain is on fire or is crashing")
    - For multiple services ("sector boom or bust")
  - Different block reward and fee structures are tested against the above
- **KPIs**:
  - Servicer NPV (KPI-1)
  - Gateway NPV (KPI-3)
  - POKT Circulating Supply (KPI-4)
  - POKT Available Supply (KPI-5)
  - Mint/burn rate (KPI-D)
  - Application relay cost per byte (KPI-6)
  - DAO value capture (KPI-10)

## Protocol Parameters to be swept

1. `RelaysToTokensMultiplier` (`RTTM`)
2. `GatewayFeePerRelay` (`GFPR`)
3. `ApplicationFeePerRelay` (`AFPR`, assumed different from `GFPR`)
4. `GatewayMinimumStake` (`GMS`)
5. `ApplicationMinimumStake` (`AMS`)
6. `DAOAllocation` (`DAL`)
7. `ValidatorAllocation` (`VAL`) 
8. `ServicerAllocation` (derived parameter, defined as 1 - `DAOAllocation` - `ValidatorAllocation`)

## Exogenous Parameters to be swept
1. Average arrival rate of the relay arrival process (cf. below): $\pi_0$
    - For a single service demanded by applications (i.e. rate is only adjusted for one particular service)
    - For multiple services demanded by applications (i.e. rate is adjusted for many services)
2. Probability of a servicer linking to a service: [params["service_linking_probability_normal"]](https://github.com/BlockScience/PocketSimulationModel/blob/main/model/config/params.py): $p_{link} \in (0,1]$
3. Number of services to be affected by the average relay arrival rate adjustment: $C$


## Exogenous Distributions that are fixed for this scenario (but may depend upon a parameter being swept)

1. The probability distribution of relays requested by an application per day: $F^a(r; \boldsymbol \pi) \in \mathcal{F}$. Probability distribution (within the set of continuous probability distributions with bounded support $\mathcal{F}$) of the number of relays per application per day (a random variable). This distribution is parameterized by a finite number of parameters $\boldsymbol \pi$, $|\boldsymbol \pi| < \infty$. _One_ of these parameters, say $\pi_0$, represents the arrival rate of relays over the course of one day. This parameter is changed halfway through each simulation (see below). 

**For this scenario, the distribution is located in the function** `submit_relay_requests_ba_gamma()` [within model/boundary_actions/application_ba.py](https://github.com/BlockScience/PocketSimulationModel/blob/main/model/boundary_actions/application.py#L116) 

## Simulation Initialization

1. Initialize a set $N_s$ of servicers
2. Initialize a set $N_c$ of services (chains)
3. Initialize a set $N_a$ of applications
4. Initialize a set $N_g$ of gateways
5. Initialize the network of applications - gateways - services - servicers
6. Initialize the number of timesteps $T$
7. Initialize the number of services affected by the relay adjustment: $C$


## Simulation routine, service demand change
Simulation runs with default behavior for $T/2$ timesteps, to 'burn in' ecosystem token flows

In timestep $T/2 + 1$:
1. Randomly draw without replacement $C$ services from $N_c$. For each of these services (or this service, if $C = 1$), adjust the relay demand by using one of the sweep values for the average arrival rate $\pi_0$. Call the resulting distribution parameter vector $\boldsymbol{\pi}^*$.
2. For each application in $N_a$:
   - For each service $c$ from the $C$ services that were affected by the relay adjustment: 
     - Draw a relay demand realization $R(c)$ from $F^a(r;\boldsymbol{\pi}^*)$
   - Draw the remaining relay demand realizations from $F^a(r;\boldsymbol \pi)$ using the default parameter values $\boldsymbol \pi$.

**Simulation then runs for the remaining $T/2$ timesteps, with the network using the new average arrival rate value for the $C$ affected services.**


## Full Grid Parameter Sweep Values (10 parameters in total)

|  Name | Sweep Values | Units |
| --- | ---| ---|
| `RTTM` | (100, 150, 200) | uPOKT/relay |
| `GFPR` | (20, 25, 30) | uPOKT/relay |
| `AFPR` | (20, 25, 30) | uPOKT/relay |
| `GMS`  | (1e5, 1.5e5, 2e5) | POKT |
| `AMS`  | (1e4, 1.5e4, 2e4)  | POKT |
| `DAL`  | (0.05, 0.10, 0.15) | % |
| `VAL`  | (0.01, 0.05, 0.10) | % |
| $\pi_0$  | (250, 25000, 250,000) ? | relays/day |
| $p_{link}$  | (0.001, 0.01, 0.1) | prob |
| $C$  | (1, 3, 5) | number |

**Total number of parameter constellations**: $3^{10} = 59,049$

**Total number of Monte Carlo runs per constellation**: ? 30

**Total number of experiments**: 1.77147 million

## Adaptive Grid Parameter Sweep Values (10 parameters in total)

|  Name | Sweep Values | Units |
| --- | ---| ---|
| `RTTM` | (100, 200) | uPOKT/relay |
| `GFPR` | (20, 30) | uPOKT/relay |
| `AFPR` | (20, 30) | uPOKT/relay |
| `GMS`  | (1e5, 2e5) | POKT |
| `AMS`  | (1e4, 2e4)  | POKT |
| `DAL`  | (0.05, 0.15) | % |
| `VAL`  | (0.01, 0.10) | % |
| $\pi_0$  | (250, 25000, 250,000) ? | relays/day |
| $p_{link}$  | (0.001, 0.01, 0.1) | prob |
| $C$  | (1, 3, 5) | number |

**Total number of parameter constellations**: $2^{7} \times 3^3 = 3,456$

**Total number of Monte Carlo runs per constellation**: ? 30

**Total number of experiments**: 103,680

## Loop algorithm (as many times as is feasible given resource/time constraints)

1. Run experiments
2. Assess KPIs via threshold inequalities/sensitivity analysis
3. Select representative 'best' constellation
4. Generate new $(min,max)$ grid sweep values for each parameter around 'best' constellation values
5. Loop--back to #1.